In [12]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

base_points_path = './base-points.csv'
transformed_points_path = './transformed-points.csv'

base_points = np.loadtxt(base_points_path, delimiter=',', skiprows=1)
transformed_points = np.loadtxt(transformed_points_path, delimiter=',', skiprows=1)

In [15]:
neighbors = NearestNeighbors(n_neighbors=1, algorithm='auto')
neighbors.fit(base_points)

distances, indices = neighbors.kneighbors(transformed_points)

In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

def icp_2d(source, target, max_iterations=100, tolerance=1e-6):
    """
    Perform the ICP algorithm (Iterative Closest Point) for 2D point clouds.

    Args:
        source (np.array): Source point cloud (Nx2)
        target (np.array): Target point cloud (Nx2)
        max_iterations (int): Maximum number of iterations
        tolerance (float): Convergence tolerance

    Returns:
        np.array: Final transformation matrix (3x3)
    """
    def closest_point(source_points, target_points):
        # Find nearest neighbors in the target point cloud for each source point
        nn = NearestNeighbors(n_neighbors=1)
        nn.fit(target_points)
        distances, indices = nn.kneighbors(source_points)
        return target_points[indices.flatten()], distances.flatten()

    # Initial transformation matrix (identity matrix for 2D)
    transformation = np.eye(3)

    # Add a column of ones to source and target points to handle translation
    source_homogeneous = np.hstack([source, np.ones((source.shape[0], 1))])
    target_homogeneous = np.hstack([target, np.ones((target.shape[0], 1))])

    prev_error = float('inf')

    for i in range(max_iterations):
        # Find the closest points in the target for the source points
        closest_points, distances = closest_point(source, target)

        # Compute centroids of both sets
        centroid_source = np.mean(source, axis=0)
        centroid_target = np.mean(closest_points, axis=0)

        # Center the points
        source_centered = source - centroid_source
        closest_centered = closest_points - centroid_target

        # Compute the covariance matrix
        H = np.dot(source_centered.T, closest_centered)

        # Singular Value Decomposition (SVD)
        U, S, Vt = np.linalg.svd(H)
        R = np.dot(U, Vt)  # Rotation matrix
        t = centroid_target - np.dot(R, centroid_source)  # Translation vector

        # Apply the transformation to the source points
        source = np.dot(source, R.T) + t

        # Compute the current error (sum of squared distances)
        mean_error = np.sum(distances) / source.shape[0]

        # Check for convergence
        if np.abs(prev_error - mean_error) < tolerance:
            print(f"Converged at iteration {i}")
            break
        prev_error = mean_error

    angle_rad = np.arctan2(R[1, 0], R[0, 0])
    angle_deg = np.degrees(angle_rad)

    print(R)

    return t, angle_deg

t, angle_deg = icp_2d(base_points, transformed_points)
print('translation', t)
print('angle_deg', angle_deg)



[[ 0.99998827  0.00484389]
 [-0.00484389  0.99998827]]
translation [-0.05104194 -0.01271559]
angle_deg -0.2775352983411378
